In [ ]:
#@title Data-AI（必ず自分の名前・学籍番号を入力すること） { run: "auto", display-mode: "form" }

import urllib.request as ur
import urllib.parse as up
Name = '\u6C5F\u6D32\u51FA\u4E95 \u5929\u9B42' #@param {type:"string"}
EName = 'Esudei Tensouru' #@param {type:"string"}
StudentID = '87654321' #@param {type:"string"}
Addrp = !cat /sys/class/net/eth0/address
Addr = Addrp[0]
url = 'https://class.west.sd.keio.ac.jp/classroll.php'
params = {'class':'dataai','name':Name,'ename':EName,'id':StudentID,'addr':Addr,
           'page':'dataai-text-8','token':'98555190'}
data = up.urlencode(params).encode('utf-8')
#headers = {'itmes','application/x-www-form-urlencoded'}
req = ur.Request(url, data=data)
res = ur.urlopen(req)

---
>テンソル(tensor)はテンション(張力:tension)に由来する\
>密度が一様の弾性球体に一方向から張力を作用させたとき体積不変のまま楕円球体へと変形するが、その時の変形との関係を表すために導入されたのがテンソルであった\
>その意味が拡大され、今のテンソルとなった\
>読みはテンソル派とテンサー派があるが、日本語なら「テンソル」が大勢で、英語なら「テンソー」が近いのではないか
---

# Tensorの扱い

PyTorchのTutorialsに従う

PyTorchについてわからないことがあれば、はやり本家の解説をまず参照するとよい

PyTorchホームページ(https://pytorch.org/)のメニューにあるDocsをクリックすると公式のドキュメントがある
- 用語や関数名など関連する内容を調べることができる
- 様々な記述例も存在するため参考にするとよい

## 基本演算

PyTorchではTensor型を用いて行列を表現する
- TensorはNumpyのndarrayと類似しているが、GPU利用による高速化が可能であるのと、後で述べるが自動微分用のパラメタが追加されている
- Tensorを作成するには、次のようにするが中身は初期化されていないことに注意する

In [ ]:
import torch
x = torch.empty(5, 3)
x

乱数で初期化するには次のようにする

In [ ]:
x = torch.rand(5, 3)
x

ゼロで初期化するには次のようにする
- torch.longは64bitの整数で与えることを意味している。

In [ ]:
x = torch.zeros(5, 3, dtype=torch.long)
x

値で初期化するには次のようにする

In [ ]:
x = torch.tensor([[0.2417, 0.2270, 0.5315],
        [0.4860, 0.9984, 0.3871],
        [0.4769, 0.8593, 0.2609],
        [0.0173, 0.1879, 0.9608],
        [0.1309, 0.9766, 0.6346]])
x

特に1で埋めるメソッドがあり、次のようにする

In [ ]:
x = x.new_ones(5, 3, dtype=torch.double)
x

xと同じサイズで異なる値を持つテンソルを生成する
- _likeがつく関数はすでに定義したxと同じサイズでテンソルを作成する
- randは平均0、分散1の正規分布で埋める

In [ ]:
x = torch.randn_like(x, dtype=torch.float)
x

大きさを知るには、sizeメソッドを呼び出す

`type(x)`ではテンソル型であることがわかるだけである

整数を要素にもつTensorと浮動小数点を要素にもつTensorをそれぞれ作成して違いを確認する
- 小文字tensorであることに注意する。

In [ ]:
x = torch.tensor([1,2,3,4,5,6])
x

In [ ]:
x.type()

In [ ]:
x = torch.FloatTensor([1,2,3,4,5,6])
x

In [ ]:
x.type()

次の大文字Tensorは注意が必要であろう

In [ ]:
x = torch.Tensor([1,2,3,4,5,6])
x.type()

In [ ]:
x = torch.randn_like(x, dtype=torch.float)
x   

In [ ]:
x.size()

足し算を行う

In [ ]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)
x+y

torchが準備するaddメソッドを用いることもできるが、やっていることは同じであるため、演算子オーバーロードされている方がわかりやすいであろう

In [ ]:
torch.add(x,y)

結果を格納するTensorを準備して、出力先を指定することができるが、直観的ではない

In [ ]:
result = torch.empty(5, 3)
torch.add(x, y, out=result)
result

次のように演算子オーバーロードを利用するのがよい
- 今何の型を使って演算しているのかがわかりにくいという意見は平成の人だから無視してよい
  - 昭和の人は型に縛られる言語を使っていない
  - 令和世代は柔軟に対応しよう
- この辺りはTensorFlowを使っている人からすると夢のような記述スタイルであろう

In [ ]:
y1 = y
y2 = y
y1.add_(x)
y2 += x
print(y1)
print(y2)

行の入手はy1[1]とすればよい

In [ ]:
y1[1]

NumPyと同様の方法で列を入手することもできる

- :でテンソル全体を指定し、その1番目の列を指定することで、１番目の列が入手できる(0オリジン)

In [ ]:
y1[:,1]

部分要素を取り出す

In [ ]:
y1[2:4,1]

view()メソッドを使って、Tensorをリサイズできる

PyTorchでテンソルを操作する関数としてtranspose, view, reshapeがある

- 基本はtransposeでTensorを転置するだけ
- view(y, x)で、y行x列のデータに変換
  - -1をどちらかに指定すると自動的に大きさから計算してくれるが、約数でないと変換できずエラーになる
  - viewはメモリ操作、つまりデータの入れ替えは行わず見え方のみ操作する関数であるため、メモリ上の並びを変えるような変換はできない
  - 例えば、transferつまり転置したデータをviewで並び替えるとメモリ上での位置の変更が伴うためエラーになる
  - これを防ぐためにviewの前にcontiguous関数を呼び出してメモリ上で要素順に並び替えてからviewすると回避できる
- reshapeは、viewと同様であるが、contiguous関数を必要としない
  - それは、データのコピーを生成するから
  - メモリは食べるけどエラーにはならない

結局自分で適切な手法を選べということ

得たい結果とその結果を得るために必要なリソースの観点があるため、様々なニーズが存在する

- 命令セットが複雑になるのは、ニーズがあるためである

- 何でも扱えるように簡単にするか、細分化して計算速度やリソースの使い方まで言及するかは言語設計の基本に影響を与えるため、言語多様性における必要悪ともいえるであろう

In [ ]:
x = torch.randn(4, 4) # 4x4の乱数行列を作成
y = x.view(16) # 16次元の行列を準備
z = x.view(-1, 8)  # -1を使うと，サイズを自動調整
print(x.size(), y.size(), z.size())

中身を確認する

In [ ]:
print(x, "\n", y, "\n", z)

要素数1のTensorにitem()メソッドを使うことで、計算履歴を含まないpythonの値として取得できる

ここで、detachとitemメソッドの違いについて説明しておく
- detachはautogradに必要な計算履歴管理を外す
  - 対象となるtorch.tensorがどのような型や要素数であっても問題ない
  - 型や総素数はなんでもよく、変化しない
- itemは要素数が1個のtorch.Tensorに限定されそれ以外はエラーとなり、スカラーとして取り出すことができる
  - 変換できるかどうかは次元数ではなく要素数で決まるため、多次元テンソルでも要素数が1個であれば変換可能
  - torch.tensorではなくなるため、結果的にdetachされている

なお、GPU変数をnumpyに取り出す際には、`.cpu().detach().numpy()`とする(後述)

In [ ]:
for i in range(3):
  print(y[[i]].item())

普通に表示するとtensor型になる

なお、```y[[i for i in range(3)]].item()```とすると、要素が1つではないためエラーとなる

In [ ]:
print(y[[i for i in range(3)]])

これは不便だと思うかもしれないが、基本的にはNumpy Arrayに変更できるため困らないはずである

なお、100種類以上のテンソル操作関数が準備されている

詳細は、
http://pytorch.org/docs/stable/torch.html
を参照のこと

- フーリエ変換、逆行列、何でもあり
- 意味を知っていて使えればよく、実装原理はここではどうでもよい

PyTorchテンソルの要素の型

テンソルの中に含める数値には、PyTorch独自のデータ型（`torch.dtypes`）が与えられているが、あるテンソルに含まれる全要素は全て同じデータ型である必要がある

以下の型が準備されているが、基本的に`torch.float`か`torch.int`しか使わないので、忘れてよい

データ型 | dtype属性への記述 | 対応するPython／NumPy（np）のデータ型
---------|-----------------|--------------------------------
Boolean（真偽値）|torch.bool|bool／np.bool
8-bitの符号なし整数|torch.uint8|int／np.uint8
8-bitの符号付き整数|torch.int8|int／np.int8
16-bitの符号付き整数|torch.int16 ／ torch.short|int／np.uint16
32-bitの符号付き整数|torch.int32 ／ torch.int|int／np.uint32
64-bitの符号付き整数|torch.int64 ／ torch.long|int／np.uint64
16-bitの浮動小数点|torch.float16 ／ torch.half|float／np.float16
32-bitの浮動小数点|torch.float32 ／ torch.float|float／np.float32
64-bitの浮動小数点|torch.float64 ／ torch.double|float／np.float64


## NumPyとの連携

### Torch TensorからNumPyへの変換

In [ ]:
a = torch.ones(5)
b = a.numpy()
b

ここから、pythonらしいマジックが入るが(これがpython嫌いを生む要素でもあるが)、numpy()はメモリを共有するため、aを変更すると、bも同時に変更される

ただし、bを変更すると、**bを変更した瞬間にこの関係は崩れ**、aを変更してもbは変更されなくなる

- この現象は、C++などにおけるCall by referenceとCall by valueの違いと同じ原理で発生する

- b = a.numpy()は、aのnumpy表示をbとする、ということで、bに実体はなく、bを呼び出すといつもa.numpy()が呼び出される

 - つまり、要するにエイリアスの関係にあり、毎回変換処理関数が呼び出される

  - そのあとb=「～」とbを変更すると、bはa.numpy()のある場所を指すのをやめて、新しく「～」というメモリ上の値を指すようになる

  - その後bを変更してもオリジナルが変更されるわけではない

python嫌いの気持ちがわかるであろう\
昨日できたことが今日できないのだから

 - 実際には=のオーバーロードである程度対応することも可能であるといえるが、言語仕様としては=しか代入がないのが問題ともいえる

なお、より実践的には次のようにする
- `to('cpu')`で、GPUではなくCPUにデータを保持する
- `detach()`で勾配計算のための付属情報を削除する
- `copy()`で不用意な書換を避ける
  - torch tensor と numpy ndarray はメモリを共有しており、どちらかを変更すると相手もそれにつられて変化する
  - これを避ける場合はtorch.clone()やnumpy.copy()もしくはdeepcopy()とする
  - 例でa.copy()はエラーとなる \
  a.to('cpu').detach().clone().numpy().copy()も無意味


In [ ]:
b = a.to('cpu').detach().numpy().copy()

In [ ]:
b=a.to('cpu').detach().clone().numpy()

In [ ]:
a.add_(1)
a

In [ ]:
b

ここでbを変更すると、aとbの一致性が崩れる

In [ ]:
b = [b[i]*2 for i in range(5)]
print(a,b)

一度崩れるともう戻らない

In [ ]:
a.add_(1)
print(a,b)

### NumPyからTorch Tensorへの変換

aに要素がすべて1で5個有するArrayとする

torch Tensorに変換するには、from_numpy()メソッドを利用する

In [ ]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)

In [ ]:
a, b

numpyで全要素に1を加えるには、ちょっと面倒な書き方になる

In [ ]:
print(a)
np.add(a, 1, out=a)
print(a)

bを表示すると、bも変化していることがわかる

In [ ]:
b

次のように記述すればよいではないか？という意見もあるであろう

In [ ]:
a += 1
a

では、bをみてみよう

同様にbも更新されていることがわかる

In [ ]:
b

さらに、次はどうだろうか

In [ ]:
a = a + 1
a

bを見てみよう

今度は更新されない

In [ ]:
b

=はメモリのコピーを生成するが、+=はコピーを生成しない、こんなややこしいことになっているが、奇天烈な日本語よりはいい

aとbのリンクが切れてしまったので時を戻そう

In [ ]:
b = torch.from_numpy(a)
b

時は戻らなかったが、悪くないだろう

次に、逆にbを変更する、もう答えはわかるであろう

In [ ]:
b+=5
b

次にaを表示する。

In [ ]:
a

きちんとaも変化している

では、代入するとどうであろうか？

In [ ]:
a = np.zeros(5)

In [ ]:
print(a)
print(b)

ということで、今度はaを変化させても、bは変化しないが、理由はこちらも同様である

つまり、メソッドを使ううちはリンクは切れないが、違うインスタンスに付け替えられるとリンクが切れて、新しい値へのリンクになる

さて、TensorとNymPy相互変換について、from_numpy()を紹介したが、これとは別にcopy()が存在する

- 生成されるデータは変わらないが、from_numpy()がメモリ共有するのに対して、copy()はメモリコピーを行う

- 同様に、にnumpy()によりtensorからnumpyに変換する場合もclone()が用意されている

- なお、PyTorchは一般にtorch.floatを利用するので、numpyでfloat32にキャストしてから利用することになる

いろいろと面倒であろうから、実践的な変換事例を示す
- ついでなので、`%%timeit`による処理速度計測の例も一緒に示すので結果の確認と処理遅延計測を行ってみよう
- なお、普通にfrom_numpyとして通常はなんら問題はない

numpyからtensorへ

In [ ]:
%%timeit
b = torch.from_numpy(a.astype(np.float32))

In [ ]:
%%timeit
b = torch.from_numpy(a.astype(np.float32)).clone()

tensorからnumpyへ

GPUメモリで操作する場合はgpuとするが、numpyはcpuでしか操作できない
- cpuにもってきて操作した場合でさらにgpuで引き継いで計算する場合は忘れずにb.to('gpu')などとしてgpuに移動させること

In [ ]:
%%timeit
c = b.to('cpu').detach().numpy()

In [ ]:
%%timeit
c = b.to('cpu').detach().numpy().copy()

ついでに、pythonのネガティブキャンペーンを続けよう

次は、`a = np.array([0,1,2])`と同じである

In [ ]:
a = np.arange(3)
a

これを転置する

In [ ]:
a.T

え？転置できない？？？

その通り、これではできない\
次のようにする

In [ ]:
a = np.array([[0, 1, 2]])
a

転置する

In [ ]:
a.T

なぜ2重配列なのだ？と思うかもしれないが、そういう仕様である
- [0, 1, 2]はメモリに順番に0, 1, 2と記載されていることを意味するので、転置されてもメモリに格納されている順序や格納形態は変わらない
- [0], [1], [2]も同様ではないか、と思うかもしれないが、これは違う
  - [0]は配列であり、メモリ上は、array([0]), array([1]), array([2])とarrayであるという識別子も記載されており、占有メモリサイズは[0, 1, 2]よりも大きい、つまり、より表現力が拡大している

- array([0, 1, 2])とは、メモリ上の表現自体が変化しており、転置に成功する

もうひとつ、`TypeError: '～' object is not callable`というエラーに遭遇したら要注意

これは、`～`というpythonのコマンドと同じ名前の変数を作ってしまった場合など発生し、初心者は結構これでハマることが多い

- 周辺の記述を間違えると、`～`という記述がコードに出てきてそれが変数と解釈される結果、このエラーが発生する

対処法は、例えば`%whos`コマンドで、使用している変数一覧をみて、問題となるpythonコマンドが変数として宣言されていないかを確認する

- 宣言されていれば、`del ～`として削除すればよい

- ぶっちゃけ、正しいコードである自信があるなら、仮想マシンをリフレッシュしてもよい

  - メニューのランタイムから、ランタイムを出荷状態にリセットを選択すればよい

## GPUとCPUの使い分け

まず、GPUが搭載されているか、どのGPUが使われているかを確認する

In [ ]:
if torch.cuda.is_available():
  print(f'GPUデバイス数： {torch.cuda.device_count()}')
  print(f'現在のGPUデバイス番号： {torch.cuda.current_device()}')
  print(f'1番目のGPUデバイス名： {torch.cuda.get_device_name(0)}')
else:
  print('GPUは使えません')

現在利用しているCPUのデバイスを取得する

In [ ]:
device = torch.device("cuda")          # デフォルトのCUDAデバイスオブジェクトを取得
device0 = torch.device("cuda:0")       # 1番目（※0スタート）のCUDAデバイスを取得
print(device)
print(device0)

テンソル計算にGPUを使う場合について以下の通りにする

- まずテンソルをGPUで生成するときはdeviceで指定する
- テンソルを移動するときはtoを使う
- 別の書き方として、
  - GPUで演算するには、演算前にcudaメソッドを使う
  - CPUで演算するには、演算前にcpuメソッドを使う

もちろんであるが、直接cpuやcudaをしてしてもよいし、省略してもよい

In [ ]:
g = torch.ones(2, 3, device=device)
g = x.to(device)
g = x.cuda(device)
f = x.cpu()
g = x.to("cuda")
f = x.to("cpu")
g = x.cuda() # GPUがセットされていて、かつGPUが獲得できていなければエラーになる

なお、テンソルではなく、学習したモデルのcpuとcudaの移動は次のようにする
- model.cuda()
  - モデルの全パラメーターとバッファーをCUDAに移行
- model.cpu()
  - モデルの全パラメーターとバッファーをCPUに移行

# 自動微分について

自動微分(Autograd)とは
- Tensorの各要素による微分を自動で行う機能
- 演算内容と計算グラフを保持し順伝播の経路を遡って勾配を計算できる

PyTorchのテンソルとAutogradにより、逆伝播の演算を自動化できる

### grad（勾配）の扱い

- Pytorch内では、torch.Tensorクラスでテンソル（多次元行列）を扱う
- .requires_gradメンバ変数で勾配をトラックするかを選択できる
  - Trueにすると重くなる
  - .backward()で勾配（微分値）を取得する
  - また、得られた勾配から誤差逆伝搬法で重みを更新する
- .bradメンバ変数に勾配データを保持する
- .detach()メンバ関数でトラック対象から外すことができる
- .requires_grad_ メンバ関数でフラグを反転させることができる。
 
各変数は、.grad_fnというプロパティ（属性）を保有し、これが実際に勾配を計算するメソッドである

In [ ]:
import numpy as np
import torch
import torch.nn as nn

## 実際の操作

テンソルを作成する

requires_grad=Falseとすると微分の対象にならず勾配としてNoneを返す

Trueとして宣言する

In [ ]:
x = torch.ones(2, 2, requires_grad=True)
x

y = x+2を計算し表示させる

In [ ]:
y = x+2
y

.grad_fnを確認する

AddBackwordと表示されるように、足し算であることを記録している

In [ ]:
y.grad_fn

関数へのポインタが示されているので実態はわからないが、足し算であることはわかる

$Y\cdot Y \cdot 3$を計算し表示する

In [ ]:
z=y*y*3
z

zの平均を計算する

In [ ]:
out = z.mean()
out

平均値である27が得られる

このZつまり平均を求める計算も勾配計算でき、MeanBackwordとして示されている

### autograd属性の反転

2x2の平均が0である乱数Tensorを作成する。

In [ ]:
a = torch.randn(2,2)
a

requires_gradを確認する

randnはデフォルトがFalseであるため、明示的にTrueとする

In [ ]:
a.requires_grad

In [ ]:
a.requires_grad_(True)
a.requires_grad

$a\cdot a$を求め、その要素の総和を計算する。

In [ ]:
b=(a*a).sum()
b

### backward()を用いて微分値を求める

まず、ここまでを振り返ると、out という値を得るために、
- 要素がすべて1の2x2行列$\boldsymbol{a}$を定義
- $\boldsymbol{y} = \boldsymbol{x}+2$を計算
- $\boldsymbol{z} = \boldsymbol{y}\cdot \boldsymbol{y} \cdot 3$を計算
- outをzの要素の平均値とする

以上の計算を行ってきた

ここで、xの変化量が、outにどれだけ影響するか？の勾配を算出する

out.backward()として、微分演算ができるようにし、実際にxによる偏微分を求める

つまり、 
${d\boldsymbol{out} \over d\boldsymbol{x}}$
を計算する。

In [ ]:
out.backward()
x.grad

答えを確認する

$$\boldsymbol{z}_i = 3(\boldsymbol{x}_i+2)^2$$
$$\boldsymbol{out} = {1 \over 4}\sum_i \boldsymbol{z}_i$$
$$\left.\boldsymbol{z}_i \right|_{x_i=1} = 27$$
である

$$
\begin{aligned}
 {\partial \boldsymbol{out} \over \partial \boldsymbol{x}_i}&={1 \over 4}\cdot{\partial (3(\boldsymbol{x}_i+2)^2)\over \partial \boldsymbol{x}_i} \\
&={3 \over 2}(\boldsymbol{x}_i+2)
\end{aligned}
$$
となる

$\boldsymbol{x}_1=1$のとき$4.5$となることがわかる

同様に、$y=w\cdot x + b$について、実際に微分値を求める
- $(x, w, b)=(1, 2, 3)$と初期化し、requires_gradをTrueとする
- $y$を求め、`y.backward()`で勾配を計算し、これを表示する

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
w = torch.tensor(2.0, requires_grad=True)
b = torch.tensor(3.0, requires_grad=True)
y = w * x + b
y.backward()
print(x.grad)  # dy/dx = w = 2
print(w.grad)  # dy/dw = x = 1
print(b.grad)  # dy/db = 1
x

$y=x^2$について計算してみると${dy \over dx} = 2x$であり、実際$x=2$のとき4になる

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
y.backward()
x.grad

なお、若干型が異なるが、次のようにしても計算できる

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = x ** 2
torch.autograd.grad(y, x)

$y=e^x$とすると、${dy \over dx}=e^x$であるから、$e^2$の値が得られる

計算グラフを構築するときは numpy の関数 numpy.exp() を使ってはいけない
- 当然であるが、テンソル計算を行う専用の関数を使う torch.exp()


In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = torch.exp(x)
y.backward()
x.grad

(利用するかどうかはともかく)三角関数も問題ない

$y=sin(x)$のとき${dy \over dx} = cos(x)$であるので、$x=\pi$のとき-1になる

In [ ]:
x = torch.tensor(np.pi, requires_grad=True)
y = torch.sin(x)
y.backward()
print(x.grad)

もっと複雑な例で試す

$y=(\sqrt{x}+1)^3$とすると${dy \over dx} = {3(\sqrt{x}+1)^2 \over 2\sqrt{x}}$となる

答えがあっているかどうかは各自で確認してほしい

In [ ]:
x = torch.tensor(2.0, requires_grad=True)
y = (torch.sqrt(x) + 1) ** 3
y.backward()
x.grad

偏微分も問題ない

$z=(x+2y)^2$とすると、
- ${\partial z \over \partial x} = 2(x+2y)$
- ${\partial z \over \partial y} = 4(x+2y)$

となることから、$(x, y) = (1, 2)$のとき、それぞれ10および20となる

In [ ]:
x = torch.tensor(1.0, requires_grad=True)
y = torch.tensor(2.0, requires_grad=True)
z = (x + 2 * y) ** 2
z.backward()
print(x.grad)  # dz/dx
print(y.grad)  # dz/dy

計算を連結させても問題ない

In [ ]:
def calc(x):
  x = x*2+1
  x = x**2
  x = x/(x+2)
  x = x.mean()
  return x
x = [1.0, 2.0, 3.0]
x = torch.tensor(x, requires_grad=True)
y= calc(x)
y.backward()
x.grad.tolist()

勾配が正しく計算できているかどうかをxを微小変化させたときのyの差を見て確認する

確認作業では特に自動微分の機能を使わないのでrequires_gradをFalseとして計算速度を上げる
  - といってもほとんど変わらないので意味はない
  - それでもやる！地球にやさしい授業にしたい


In [ ]:
delta = 0.001  #変化量
x = [1.0, 2.0, 3.0]
x = torch.tensor(x, requires_grad=False)
y = calc(x).item()

x_1 = [1.0+delta, 2.0, 3.0]
x_1 = torch.tensor(x_1, requires_grad=False)
y_1 = calc(x_1).item()

x_2 = [1.0, 2.0+delta, 3.0]
x_2 = torch.tensor(x_2, requires_grad=False)
y_2 = calc(x_2).item()

x_3 = [1.0, 2.0, 3.0+delta]
x_3 = torch.tensor(x_3, requires_grad=False)
y_3 = calc(x_3).item()

# 勾配の計算
grad_1 = (y_1 - y) / delta
grad_2 = (y_2 - y) / delta
grad_3 = (y_3 - y) / delta

print(grad_1, grad_2, grad_3)

殆ど値が一致していることがわかるであろう

PyTorchの機能や中身の一部であるが核心が理解できたであろう

# 実際のデータの取り込み

実際にはコード中でデータを直接記述することはなく、外部からデータを読み込むことが殆どであろう

また、データはcsvなどの形式から取り込むであろうが、学習においては、準備されているデータセットを用いることも多いであろう

- PyTorchも様々なデータセットを準備しており、詳細はPyTorchのDocsを参照するとよい

ここでは、PyTorchが準備するデータセット、さらにPyTorchが備えるDataLoader、またPyTorchにおけるミニバッチ法の実装例を示す

今回も手書き文字認識であるが、MNISTと呼ばれる、より本格的なデータを用いる

## DataLoaderを用いたミニバッチ法

既に簡単に触れたが、PyTorchが準備するDataLoaderを利用するとデータの読み込みおよびミニバッチ法の実装が容易になる

次の例のように、`torchvision.datasets`を使って最初に手書き文字のデータを読み込む

- `torchvision.datasets`には様々なデータセットが準備されている

`transform`をimportすることで、前処理としてのデータ変換を行う`transforms.ToTensor()`が利用できるようにする

- PyTorchはテンソル型のデータしか扱えないため、変換するために必要
- なぜテンソル型にする必要があるかは既に説明済み

そして**DataLoader**を用いてデータ分割とバッチ生成を行う

- scikit-learnのtrain_test_split関数と目的は同じ
  - PyTorchは、モデル側もバッチに対応してまとめて入力できるようになるため、モデル側の記述の修正が不要

- DataLoaderではバッチサイズを指定してデータ分割を行うため応用しやすい

- ここではバッチサイズを大きめの256と設定しているが、GPU搭載マシンである場合はこの程度の数でも問題ない

  - 大きめの数を指定することで、GPUの能力を発揮できるようになる
  - ただし、まとめて逆伝播の勾配計算も行われるため、学習の進み方、精度などに影響することに注意する

- 訓練用データの取り出しには、毎回異なるデータをランダムに取り出してバッチを生成させるため、shuffleをTrueにしている

- テスト用データの取り出しは、そのような必要はないため、Falseである

次の実装例では、PyTorchが提供するデータセットを利用するので問題ないが、自分でデータセットを準備する場合、DataLoaderが読めるのは入力とラベルがセットになったデータである
  - 既にscikit-learnが準備するデータセットを用いた例において、TensorDatasetを使ってセットを作ることを学習済みである

これには、次の命令を利用して事前に作成しておく必要がある
- `ds = TensorDataset(x, y)`

なお、testデータの方は、特にミニバッチ法を取り入れる意味はないため、全数指定している

- ミニバッチ法を採用しても、悪いということはない

  - つまり`batch_size=len(mnist_test)`の部分をtrainと同様にしても問題はない

In [ ]:
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader
# 訓練データを取得
mnist_train = MNIST("./mydata", # 保存先フォルダの指定
                    train=True, download=True,
                    transform=transforms.ToTensor())
# テストデータの取得
mnist_test = MNIST("./mydata", # 保存先フォルダの指定
                   train=False, download=True,
                   transform=transforms.ToTensor())
print("訓練データの数:", len(mnist_train), "テストデータの数:", len(mnist_test))
# DataLoaderの設定
batch_size = 256
train_loader = DataLoader(mnist_train, 
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(mnist_test,
                         batch_size=len(mnist_test),
                         shuffle=False)

## モデル構築

前回は直接Sequentialによりモデルを構築したが、今回はより実践的に`nn.Module`モジュールを継承したクラスとしてモデルを構築する

`.cuda()`により、モデルの計算はGPU上で行われる

- Google ColaboratoryはGPUに対応している

- メニューの編集、ノートブックの設定から、GPUを利用するのチェックが入っていることを確認しておく

`__init__`つまりコンストラクタでトポロジを指定する

- 親クラスのコンストラクタを呼び出すのを忘れないように

forwardメソッドで活性化関数関数を指定する

ネットワーク全体で見て、入力は画素数、出力は10個の数字である

- 今回利用するMNISTの画像データの画素数は784である
  - scikit-learnの手書き文字認識が$8\times 3=64$であったのに対して$28 \times 28 = 784$と大幅に増えている

  - 画素数も多いため、隠れ層のノード数も1024とかなり大きくしている
  - ただし層の数は少なめで、それほど精度は期待できないかもしれない

ここでは、$28 \times 28$の2次元画像構成を単純に$784$の1次元構成に変換するため、並び替えは伴わず、高速で効率の良いviewを利用する

- transferでも動作し、この程度ならば殆ど変わらない


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(784, 1024)  # 全結合層
    self.fc2 = nn.Linear(1024, 512)
    self.fc3 = nn.Linear(512, 10)
  def forward(self, x):
    x = x.view(-1, 784)  # バッチサイズ×入力の数
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x
net = Net()
net.cuda()  # GPU対応
print(net)

## 学習

モデルを訓練する

**DataLoaderを使い、順次ミニバッチを取り出して訓練と評価を行う**

1エポックの中で何度もミニバッチを使って訓練される

なお、評価もミニバッチができるような記述であるが、全数を一気に取得するので、実際はミニバッチではない

In [ ]:
from torch import optim
# 交差エントロピー誤差関数
loss_fnc = nn.CrossEntropyLoss()
# SGD
optimizer = optim.SGD(net.parameters(), lr=0.01)
# 損失のログ
record_loss_train = []
record_loss_test = []
# 学習
for i in range(20):  # 20エポック学習
  net.train()  # 訓練モードへ
  loss_train = 0
  for j, (x, t) in enumerate(train_loader):  # ミニバッチ(x,t)を取り出す
    x, t = x.cuda(), t.cuda()  # GPUのメモリに配置する
    y = net(x)
    loss = loss_fnc(y, t)
    loss_train += loss.item()  # ミニバッチなので、誤差を蓄積させていく
    optimizer.zero_grad() # RNNではためることもあるが普通は初めに勾配を0初期化必須
    loss.backward() # 逆伝播してパラメタを計算
    optimizer.step() # 計算した値でパラメタを更新
  loss_train /= j+1  # ループから抜けたらロスの平均を計算
  record_loss_train.append(loss_train)
  net.eval() # 評価モードへ
  loss_test = 0
  for j, (x, t) in enumerate(test_loader):  # ミニバッチ(x,t)の取り出し、trainと全く同じ
    x, t = x.cuda(), t.cuda()
    with torch.no_grad():
      y = net(x)
    loss = loss_fnc(y, t)
    loss_test += loss.item()
  loss_test /= j+1
  record_loss_test.append(loss_test)
  print("Epoch:", i, "Loss_Train:", loss_train, "Loss_Test:", loss_test)

## 誤差の推移
訓練データ、テストデータで誤差の推移をグラフ表示する

TestよりもTrainの方が誤差が大きく出ているが、これはミニバッチにより複数の訓練の平均のロスと訓練後のロスを比較しているためである
- これは、仮にTestを全数ではなくミニバッチにしたとしても見られる現象である

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(len(record_loss_train)), record_loss_train, label="Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label="Test")
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.show()

## 正解率
モデルの性能を把握するため、テストデータ使い正解率を測定する

In [ ]:
correct = 0
total = 0
net.eval()
for i, (x, t) in enumerate(test_loader):
  x, t = x.cuda(), t.cuda()  # GPU対応
  x = x.view(-1, 784)
  with torch.no_grad():
    y = net(x)
  correct += (y.argmax(1) == t).sum().item()
  total += len(x)
print("正解率:", str(correct/total*100) + "%")

## `model.eval()`と`torch.no_grad()`について

### `model.eval()`

`.eval()`はevalモードでの動作でBatch Normalization(Normを含むnn.Moduleの層).やDropoutなどが評価用として動作する

- これらの方法を用いた時の評価では評価結果に影響するため必須

### `torch.no_grad()`

一般に、
```
with torch.no_grad:
  modelの評価コード
```
といった利用の仕方となる

- PyTorchは計算履歴を変数に保存するため、GPUを利用する場合、GPUメモリを浪費し、実行できなくなる場合がある
  - train時やforward計算時に勾配計算用パラメータを保存することでbackward計算を高速化するautograd機能のため

- autogradを無効化し、メモリ使用量を減らして計算速度を向上させる
  - 必須ではなく、無くても評価結果に影響しない
  - メモリサイズがおよそ1/5になるため、GPUでは5倍といった大きなバッチを組むことができる
  - CPUはメモリが比較的十分にあり、全てのモデルが基本メモリ上で動作させるため、その効果は微妙になるといえる

- 計算履歴を保存する必要がない場合、例えばbackwordが必要ない検証などは、保存しない設定で計算するとよい

- optimizer.zero_grad()を呼び出す必要がなくなる
  - そもそも評価時なので不必要

### `torch.no_grad`の具体例

ついでに、```with torch.no_grad():```とするのと```torch.set_grad_enabled(False)```とするのが違うことも示す

訓練時は勾配計算が必要であるが、評価時は不要である

その時、
```
model.eval();torch.set_grad_enabled(False)
```
とすると、計算記録の追加は行われないが、過去の計算記録は残っているので、処理速度は速くなるが、メモリ占有量は変わらない

次に、
```
with torch.no_grad():
  model.eval()
```
とすると、記録すらもなくなるため、メモリ占有量も減る

> withの用法について
>
> withは、開始と終了がセットになった処理を行うという一般的な説明がされているが、これは、コンストラクタはともかく、デストラクタの呼び出しが曖昧なガベージコレクト型言語のpythonについて、明示的なデストラクタを呼び出すことを意味している
>
> ```
with open('filename', 'w', opener=opener) as f:
    print('test', file=f)
```
> などとして使う
> 
> printが実行されるとデストラクタが明示的に呼び出され、fのメソッドとしてのデストラクタ、つまりcloseが呼び出されるため実質close不要

### パラメータの変更について

それ以外に、例えば、学習途中で、強制的に重みを変える、消失しそうだから、2倍にしたいとする(そういうことをする意味があるかどうかはさておき)

```
for param in model.parameters():
  param.data *= 2.0
```
これは、lossの勾配を計算する際にエラーになる

つまり、2倍するという計算の履歴すらも記録されてしまうからである

次のようにするとよい
```
with torch.no_grad():
  for param in model.parameters():
    param.data *= 2.0
```
自ら正規化規則を記述するぐらいの気概がある場合は参考になるかもしれない

# PyTorchにおけるクロスエントロピーについて

PyTorchのCrossEntropyLoss関数には癖があるのでここで説明する

まず、PyTochにおける正解となるターゲットの記述の仕方に注意する
- One-hotで正解が3(ここでは0から数えて2)であるoutputと、正解ラベルである3(これも2)を格納したtargetを準備する

まず、どちらも、**配列の1つの要素として**、one-hotの値や2が格納されていることに注意する
- 複数の演算を同時に行う必要から、配列の要素として扱う
  - 今は一つであるが、複数入れるとそれぞれで計算してくれる
- このあたりの配列の扱いに慣れていないと、すぐにパニックに陥る
- 動いている例を見つけたら、typeやshapeにより型をきちんとしらべること

もう一つ、outputはFloatTensor、targetはLongTensorであることに注意すること
- ニューラルネットワークの出力は通常FloatTensorで、正解ラベルは通常整数であることから、このようになっている
  - 親切からそうなっているが、何も考えないで用いるとトラブルの要因の一つになりうる

なお、FloatTensorを用いているが、`torch.Tensor([[0,0,1]]).to(dtype=torch.long)`というようにtoで変換してもよく、floatであれば`dtype=torch.float`となる


In [ ]:
import torch
import torch.nn as nn
import numpy as np

output = torch.FloatTensor([[0,0,1]])
target = torch.LongTensor([2])
print(output, target)

この場合、定義通りクロスエントロピーロスを計算しようとすると、次のようになる

但し、極端な出力のため、$log 0$は計算できず、そのまま省略している
$$Loss(p,q)=-\sum_i{p_i log q_i}$$

必要な箇所だけ計算すると、

In [ ]:
-(1*np.log(1))

となり、つまり0である

しかしながら、PyTorchのクロスエントロピーは異なる値となる

In [ ]:
crossentropy = nn.CrossEntropyLoss()
crossentropy(output, target)

なお、本来は`[0,0,1]`といった美しい確率分布になるような出力値がNNから得られることはまずない。
- これを確率とするには、一度Softmaxを介さなければならない

次のような例でも問題ないことを考えれば、より分かりやすいのではないだろうか

outputは、`[0,0,1]`としているが、別に次のような値でも構わない
- ニューラルネットワークからの出力は0から1の値とは限らず、理論的には実数域すべてを想定できる
- 配列の最後の要素がとびぬけていると値が小さくなる。

In [ ]:
output2 = torch.FloatTensor([[2.0,35.2,-13.7]])
print(crossentropy(output2, target))
output3 = torch.FloatTensor([[2.0, -13.7, 35.2]])
print(crossentropy(output3, target))

さて、PyTorchのCrossEntropyLoss()の出力値が、実際のクロスエントロピーの定義に基づく式の演算結果と異なるのは、PyTorchのCrossEntropyLoss()は、Softmax()とNLLLoss()の合成で計算されるためである

確認しよう
- まず、Softmaxの結果だけみてみよう

In [ ]:
softmax = nn.Softmax(dim=1) # dim=1を忘れるとワーニングが出る
softmax(output)

この結果のlogを取り、NLLLossを計算する

NLLLossは、この例では$$-x[class]$$を計算するだけの関数
- 実際には、まとめて計算するため複数の要素が投入されるが、それらすべての平均を計算してくれる関数
- ミニバッチの場合、まとめて計算し、その平均でパラメータを更新するが、そのためにある関数

In [ ]:
nllloss = nn.NLLLoss()
nllloss(torch.log(softmax(output)), target)

このように答えが一致した

つまり、もともとのクロスエントロピーの定義に基づけば、`nsloss(torch.log(output), target)` となる

しかしながら実際の利用において、この形単独で利用するケースは限られることからPyTorchではsoftmax関数が内包されている

さて、CrossEntropyを利用するのと、Softmax+log+NLLとするのと同じだから、どちらでもよいではないか？ということになりそうであるが、そうではない。

専用関数は次のような恩恵に預かることができる

- exp-normalize trickの恩恵を最大化するのは、CrossEntropyで一気にやる方法であること
- 内部で最適化されていること

この、exp-normalize trickについて簡単に説明する

普通にsoftmaxを定義通り計算すると、
$$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$$
となるが、$e^x$が大きすぎ、計算機上での計算において、計算結果がオーバーフローする可能性がある

そこで、値を小さくして計算する手法として、
$$
e^{x_i}\cdot e^{-b}\cdot e^{b}=e^{x_i-b}\cdot e^{b}
$$
となることから、
$$
\text{softmax}(x_i) = \frac{e^{x_i-b}e^{b}}{\sum_j e^{x_j-b}e^{b}} = \frac{e^{x_i-b}}{\sum_j e^{x_j-b}}
$$
として計算しても同じ答えとなることがわかる

PyTorchは、これらの処理も自動的に行っている

log_softmaxについても同様で、log-sum-expと呼ばれる、名前だけ変わってやっていることは変わらないテクニックがある

$$
\begin{align}
\text{log_softmax}(x_i) &= \log\left(\frac{e^{x_i}}{\sum_j e^{x_j}}\right)\\
&= x_i - \log \left( \sum_j e^{x_j} \right) \\
&= x_i - \log \left( \sum_j e^{x_j - b}e^b \right) \\
&= x_i - \log \left( \sum_j e^{x_j - b} \right) - b \\
\end{align}
$$

とにかく、複合関数を使わないと、予期せぬ落とし穴にはまることがある

# Pytorchの活性化関数

PyTorchでは、以下の活性化関数が用意されている

ELU, Hardshrink, Hardtanh, LeakyReLU, LogSigmoid, MultiheadAttention, PReLU, ReLU, ReLU6, RReLU, SELU, CELU, GELU, Sigmoid, Softplus, Softshrink, Softsign, Tanh, Tanhshrink, Threshold, Softmin, Softmax, Softmax2d, LogSoftmax, AdaptiveLogSoftmaxWithLoss

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
def drawGraph(x, y, y_dash, title):
  fig = plt.figure(figsize=(12, 4))
  ax1 = fig.add_subplot(1, 2, 1)
  ax2 = fig.add_subplot(1, 2, 2)
  ax1.plot(x, y)
  ax2.scatter(x, y_dash, s=1, color='orange')
  ax1.set_title(title)
  ax2.set_title(title+' Gradient')
  plt.show()
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.relu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'ReLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.hardtanh(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'HardTanh')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.relu6(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'ReLU6')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.elu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'eLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.selu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'SeLU')
x=torch.linspace(-1, 1, 1000, dtype=torch.float, requires_grad=True)
y=F.celu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'CeLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.leaky_relu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'LeakyReLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.prelu(x, torch.tensor([0.25]))
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'PReLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.rrelu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'RReLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float)
y=F.glu(x)

# xを2つに分解する
x1=torch.linspace(-6, 0, 500, dtype=torch.float, requires_grad=True)
x2=torch.linspace(0, 6, 500, dtype=torch.float, requires_grad=True)
y=torch.mul(x1, torch.sigmoid(x2))
z=y.sum()
z.backward()
#x1 x2 で自動微分
z_dash1=x1.grad
z_dash2=x2.grad
z_dash3=torch.add(z_dash1, z_dash2)
drawGraph(x2.detach().numpy(), y.detach().numpy(), z_dash3.detach().numpy(), 'GLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.gelu(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'GeLU')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.logsigmoid(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'LogSigmoid')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.hardshrink(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'HardShrink')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.tanhshrink(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'TanhShrink')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.softsign(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'Softsign')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.softplus(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'Softplus')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.softmin(x,  0)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'Softmin')
x=torch.linspace(-6, 6, 200, dtype=torch.float)
y=F.softmax(x)
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.softshrink(x)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'SoftShrink')
x=torch.linspace(-6, 6, 1000, dtype=torch.float, requires_grad=True)
y=F.log_softmax(x, 0)
z=y.sum()
z.backward()
z_dash=x.grad
drawGraph(x.detach().numpy(), y.detach().numpy(), z_dash.detach().numpy(), 'LogSoftmax')


# Pytorchの最適化手法

次のような最適化手法があり、その基本は既に学習済である
- SGD : torch.optim.SGD
- Adagrad : torch.optim.Adagrad
- RMSprop : torch.optim.RMSprop
- Adadelta : torch.optim.Adadelta
- Adam : torch.optim.Adam
- AdamW : torch.optim.AdamW

これらについて、関数$f(x,y)=x^2+y^2$ 平面上での最適化過程を比較する
- なお次のようなグラフであり、この底へ向かわせる

In [ ]:
def func(x, y):
  return x**2+y**2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
x = y = np.linspace(-10, 10)
X, Y = np.meshgrid(x, y)
Z = func(X, Y)
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(1, 1, 1, projection='3d')  # 3D描画機能を持ったサブプロット作成
ax.plot_surface(X, Y, Z)   # 曲面を描画した3Dグラフを表示

In [ ]:
import torch
from torch import optim
import matplotlib.pyplot as plt
import numpy as np
# 次の全ての最適化手法を試すため名前をリストに登録
classes = ['SGD', 'Adagrad', 'RMSprop', 'Adadelta', 'Adam', 'AdamW']
x_list = {}
y_list = {}
f_list = {}
optimizers = {}
# それぞれの最適化手法毎に配列を準備
for key in classes:
  x_list[key] = []
  y_list[key] = []
  f_list[key] = []
# 各最適化手法を用いて実際に最適化を行う
for key in classes:
  print("Optimizer:"+key)
  # 初期値(最適化を行う最初の場所)を統一しておく
  x = torch.tensor(-75.0, requires_grad=True)
  y = torch.tensor(-10.0, requires_grad=True)
  params = [x, y]
  # 学習率（SGDはいきなり最適に向かってしまう好都合な条件のため0.1とする）
  optimizers['SGD'] = optim.SGD(params, lr=0.1)
  optimizers['Adagrad'] = optim.Adagrad(params, lr=1.0)
  optimizers['RMSprop'] = optim.RMSprop(params, lr=1.0)
  optimizers['Adadelta'] = optim.Adadelta(params, lr=1.0)
  optimizers['Adam'] = optim.Adam(params, lr=1.0)
  optimizers['AdamW'] = optim.AdamW(params, lr=1.0)
  for i in range(5000): # 繰り返し
    # Autogradを用いて勾配を求めてoptimizerで更新する
    optimizers[key].zero_grad()
    outputs = func(x, y)
    outputs.backward()
    optimizers[key].step()
    x_list[key].append(x.item())
    y_list[key].append(y.item())
    f_list[key].append(outputs.item())
# 結果のグラフ作成
fig = plt.figure(figsize=(15, 10))
for i, key in enumerate(classes):
  ax = fig.add_subplot(2, 3, i+1)
  ax.scatter(x_list[key], y_list[key])
  ax.plot(-75, -10, '+')
  ax.set_title(key, fontsize=16)
  ax.set_xlabel('x', fontsize=16)
  ax.set_ylabel('y', fontsize=16)
  ax.set_xlim(-80, 80)
  ax.set_ylim(-15, 15)
plt.tight_layout()
fig.show()

見方ですが、丁度先に示した3Dの図を上から見た時の移動を示している
- (x,y)=(0,0)が最小値でここに向かう過程を示している

## SGD（Stochastic Gradient Decent : 確率的勾配降下法）

代表的な最適化手法で、求めた勾配方向に求めた大きさだけパラメータを更新する

Pytorchでは、MomentumといったSGCの改良アルゴリズムを利用する場合、`torch.optim.SGD()`のパラメータに指定する
- パラメータには momentum、 dampening、 nesterovなどを指定できる

`torch.optim.SGD(params, lr=<required parameter>, momentum=0, dampening=0, weight_decay=0, nesterov=False)`

欠点：複雑な状況では最適化が進みにくくなる

## Adagrad（Adaptive Gradient Algorithm）

学習率を学習過程の中で更新する手法
- パラメータの更新度合いで次の学習率を各パラメータ毎に調整
  - 大きく更新されたパラメータの学習率をより小さく調整
  - 大きな勾配の影響が小さくなり、効率的に最適化を行う

欠点：学習が進むと学習率が小さくなり更新されなくなる

## RMSprop（Root Mean Square propagation）

勾配情報をAdagrad同様に記憶するが、古い情報を落として、新しい勾配情報がより反映されるように工夫されている
- 学習が進んでも学習率の調整を適切に行う

## Adadelta

AdagradやRMSpropがパラメータを勾配の単位で更新する点を改良している
- 過去の更新量の移動平均を過去の勾配の移動平均で割り、さらに現在の勾配を掛けた値を更新量とする
- つまり、常に同じ単位で更新される
- 学習率の設定が不要

欠点：収束がやや遅い

## Adam（Adaptive Moment Estimation）

MomentumとAdagradを併せ持つ

蛇行して効率が悪いように見えるかもしれないが、複雑な形状ではよりよい方向を適切に探し出すことができる場合がある

## AdamW

Adamの Weight decay（重み減衰）に関する式に変更を加えている
- 損失関数計算時およびパラメータ更新時にL2 正則化項を追加
- Adamよりも適切な Weight decay を与える


# 課題8 (TorchTensor)

これまで同様にタイトル付のノートブックとして提出しなさい

**問題1**

次のようなPyTorchテンソル t を作成しなさい
`print(t)`とすると、
```
tensor([[[  1,   2],
         [  4,   5],
         [  7,   8]],

        [[ -1,  -2],
         [ -4,  -5],
         [ -7,  -8]],

        [[ 10,  20],
         [ 40,  50],
         [ 70,  80]],

        [[-10, -20],
         [-40, -50],
         [-70, -80]]])
```
と返す

**問題2**

tの大きさをsizeメソッドを使って確認しなさい

**問題3**

`print(t[:,-1,:])`
の出力結果を記すと共に、なぜその結果になるかを説明しなさい

**問題4**

`print(t[:-1:])`
の出力結果を記すと共に、なぜその結果になるかを説明しなさい

**問題5**

問題6に対して、`t[]`の中に**数字を2箇所にだけ書き足す**ことで同じ出力結果が得られるようにしなさい

要するに問題5は、省略形であるので、省略なしに記しなさいということである
- 単純に何の省略か？という問題にすると答えが複数現れるため、あえてこのような問いにしている

**問題6**

$y = 2x^3-3x^2-12x+1$を求める関数f(def文で、def f(x):などとする)を定義し、$x=1$の時の導関数の値をPyTorchのautograd機能を利用して計算しなさい

また、手計算で結果が正しいことを確認しなさい